In [1]:
import syft as sy
import numpy as np
from syft.core.node.worker import Worker
from syft.core.node.new.user_code import SubmitUserCode
from syft.core.node.new.policy import SubmitUserPolicy

/home/teo/anaconda3/envs/PySyft/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft


In [2]:
worker = Worker()

> Starting Worker: Priceless Malik - 1c802bd4fe1d47e3994c689f71d4a335 - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.DataSubjectMemberService'>]


In [3]:
client = sy.login(node=worker, email="info@openmined.org", password="changethis")

Logged into Priceless Malik as <info@openmined.org>


In [4]:
from pydantic import BaseModel
from typing import List, Dict, Any

class A(BaseModel):
    class Config:
        arbitrary_types_allowed = True

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(**kwargs)
        print(self.__post_init__)
        print(args)
        print(kwargs)
        self.__post_init__()
        
    def __post_init__(self):
        pass
    
    def update() -> None:
        raise NotImplementedError
    
class B(A):
    init_args: Dict[str, Any]={}
    kwargs: Dict[str, Any]={}

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(**kwargs)
        self.init_args = args
        self.kwargs = kwargs

    
class C(B):
    n_calls: int
    downloadable_output_args: List[str]
    
    
    
# B()     
c = C(n_calls=1, downloadable_output_args=[])
c.kwargs

<bound method A.__post_init__ of C(init_args={}, kwargs={}, n_calls=1, downloadable_output_args=[])>
()
{'n_calls': 1, 'downloadable_output_args': []}


{'n_calls': 1, 'downloadable_output_args': []}

In [5]:
c.__post_init__

<bound method A.__post_init__ of C(init_args=(), kwargs={'n_calls': 1, 'downloadable_output_args': []}, n_calls=1, downloadable_output_args=[])>

In [6]:
# from syft.core.common.serde.serializable import serializable
from syft.core.node.new.new_policy import ExactMatch, OutputPolicyExecuteOnce, OutputPolicy, CustomOutputPolicy
from syft.core.node.new.action_object import ActionObject
from syft.core.node.new.serializable import serializable

@serializable(recursive_serde=True)
class RepeatedCallPolicy(CustomOutputPolicy):
    def __init__(self, n_calls=1, downloadable_output_args=[]):
        super().__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_args
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, action_object) -> Dict[Any, Any]:
        results_dict = action_object.syft_action_data        
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                # results_dict[output_arg] = results_dict[output_arg].set_permission(READ, self.my_verify_key)
                results_dict[output_arg] = results_dict[output_arg]

            self.state["counts"] += 1
        else:
            return None
            
        return results_dict
    
    

In [7]:
policy = RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y'])
print(policy.init_args, policy.init_kwargs)
obj = ActionObject.from_obj({'y': [1,2,3]})
policy.apply_output(obj)

() {'n_calls': 1, 'downloadable_output_args': ['y']}


{'y': [1, 2, 3]}

In [8]:
ser = sy.serialize(policy, to_bytes=True)
de_policy = sy.deserialize(ser, from_bytes=True)
de_policy == policy

False

In [9]:
de_policy.state

{'counts': 1}

In [10]:
import numpy as np
from syft.core.node.new.twin_object import to_action_object

x = np.array([1,2,3])
x_pointer = to_action_object(x)
client.api.services.action.save(x_pointer)

<class 'syft.core.node.new.response.SyftSuccess'>: <class 'syft.core.node.new.numpy.NumpyArrayObject'> saved

In [11]:
@sy.syft_function(
    input_policy=ExactMatch(x=x_pointer),
    # input_policy_init_args={'inputs':{'x':4}},
    # output_policy=SingleExecutionExactOutput,
    # output_policy_init_args={},
    output_policy=RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y']),
)
def func(x):
    return {"y": x}


SubmitUserPolicy
{'n_calls': 1, 'downloadable_output_args': ['y']}
('x',)


In [12]:
input_policy=ExactMatch(x=x_pointer)

In [17]:
input_policy.node_uid

ValueError: "ExactMatch" object has no field "node_uid"

In [14]:
client.api.services.code.request_code_execution(func)

<class 'syft.core.node.new.response.SyftError'>: Exception calling code.request_code_execution. "ExactMatch" object has no field "node_uid"

In [15]:
client = sy.login(node=worker, email="info@openmined.org", password="changethis", cache=False)

Logged into Priceless Malik as <info@openmined.org>


In [16]:
request = client.notifications[0].link
func = request.changes[0].link
result = func.unsafe_function(x=x_pointer)
# func.output_policy
final_result = request.accept_by_depositing_result(result) 
final_result

IndexError: list index out of range

In [ ]:
res = client.api.services.code.func(x=x_pointer)
# print(res)
# print(type(res))

output_policy_state <class 'bytes'>


In [ ]:
policy = client.api.services.policy.get_all()[0]

In [ ]:
client.id

<UID: 4c5dc2d254a1468784643b0ee765cdfd>

In [ ]:
from syft.core.node.new.linked_obj import LinkedObject 
from syft.core.node.new.policy import UserPolicy
from syft.core.node.new.policy_service import PolicyService

policy_linked = LinkedObject.from_obj(policy, service_type=PolicyService ,node_uid=client.id)

In [ ]:
policy_linked

```python
class LinkedObject:
  id: str = 1a38fce092a440f58ee38bc108b09ba4
  node_uid: str = 4c5dc2d254a1468784643b0ee765cdfd
  service_type: str = <class 'syft.core.node.new.policy_service.PolicyService'>
  object_type: str = <class 'syft.core.node.new.policy.UserPolicy'>
  object_uid: str = 186aff516782421ab982f8198e264ad5

```

In [ ]:
output_policy = client.api.services.policy.get_all()[0]


In [ ]:
client.api.services.policy.get_all()

,type,id
0,syft.core.node.new.policy.UserPolicy,186aff516782421ab982f8198e264ad5


In [ ]:
@sy.syft_function(
    input_policy=ExactMatch(x=x_pointer),
    # input_policy_init_args={'inputs':{'x':4}},
    # output_policy=SingleExecutionExactOutput,
    # output_policy_init_args={},
    output_policy=output_policy.id,
    output_policy_init_args={"n_calls":1, 'downloadable_output_args':['y']},
    outputs=["y"]
)
def func_plus_one(x):
    return {"y": x + 1}


UserPolicy
('x',)


In [ ]:
client.api.services.code.request_code_execution(func_plus_one)

<class 'syft.core.node.new.response.SyftError'>: Exception calling code.request_code_execution. 'UserPolicy' object has no attribute 'is_ok'

In [ ]:
client.api.services.policy.get_all()[0]

```python
class UserPolicy:
  id: str = 186aff516782421ab982f8198e264ad5
  node_uid: str = 4c5dc2d254a1468784643b0ee765cdfd
  user_verify_key: str = bd2ba16cbdad6886804f76c855f6aca232517b12c6fcb13a51bf782304f1ce19
  raw_code: str = "from .user_code import CustomOutputPolicy
@serializable(recursive_serde=True)
class RepeatedCallPolicy(CustomOutputPolicy):
    # from syft.core.node.new.action_object import ActionObject
    __canonical_name__ = "RepeatedCallPolicy"
    from syft.core.node.new.syft_object import SYFT_OBJECT_VERSION_1

    __version__ = SYFT_OBJECT_VERSION_1
    from typing import Dict
    from typing import Any
    n_calls: int
    downloadable_output_args: List[str]
    state: Dict[Any, Any] = {}
    # _init_args: Dict[str, Any]
    # _kwargs: Dict[str, Any]
    
    __attr_allowlist__ = [
        "n_calls",
        "downloadable_output_args",
    ]
    
    # __fields_set__ = [
    #     "n_calls",
    #     "downloadable_output_args",
    # ]
    
    def __init__(self, n_calls=1, downloadable_output_args=[]):
        super().__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_args
        self.state = {"counts": 0}

    # def __repr__(self) -> str:
    #     return f'RepeatedCallPolicy - {self.n_calls}/{self.downloadable_output_args}/{self.state}'

    # def update() -> None:

    # def __post_init__(self, *args, **kwargs) -> None:
    #     self._init_args = args
    #     self._kwargs = kwargs

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, action_object) -> Dict[Any, Any]:
        results_dict = action_object.syft_action_data        
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                # results_dict[output_arg] = results_dict[output_arg].set_permission(READ, self.my_verify_key)
                results_dict[output_arg] = results_dict[output_arg]

            self.state["counts"] += 1
        else:
            return 
            
        return results_dict
"
  parsed_code: str = "from .user_code import CustomOutputPolicy
@serializable(recursive_serde=True)
class RepeatedCallPolicy(CustomOutputPolicy):
    # from syft.core.node.new.action_object import ActionObject
    __canonical_name__ = "RepeatedCallPolicy"
    from syft.core.node.new.syft_object import SYFT_OBJECT_VERSION_1

    __version__ = SYFT_OBJECT_VERSION_1
    from typing import Dict
    from typing import Any
    n_calls: int
    downloadable_output_args: List[str]
    state: Dict[Any, Any] = {}
    # _init_args: Dict[str, Any]
    # _kwargs: Dict[str, Any]
    
    __attr_allowlist__ = [
        "n_calls",
        "downloadable_output_args",
    ]
    
    # __fields_set__ = [
    #     "n_calls",
    #     "downloadable_output_args",
    # ]
    
    def __init__(self, n_calls=1, downloadable_output_args=[]):
        super().__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_args
        self.state = {"counts": 0}

    # def __repr__(self) -> str:
    #     return f'RepeatedCallPolicy - {self.n_calls}/{self.downloadable_output_args}/{self.state}'

    # def update() -> None:

    # def __post_init__(self, *args, **kwargs) -> None:
    #     self._init_args = args
    #     self._kwargs = kwargs

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, action_object) -> Dict[Any, Any]:
        results_dict = action_object.syft_action_data        
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                # results_dict[output_arg] = results_dict[output_arg].set_permission(READ, self.my_verify_key)
                results_dict[output_arg] = results_dict[output_arg]

            self.state["counts"] += 1
        else:
            return 
            
        return results_dict
"
  signature: str = (downloadable_output_args)
  class_name: str = "RepeatedCallPolicy"
  unique_name: str = "user_func_RepeatedCallPolicy_bd2ba16cbdad6886804f76c855f6aca232517b12c6fcb13a51bf782304f1ce19_c74e97305b95fd6c1a37bbfe65948a8577b0ade967a3fffab5a8d400757c6617"
  code_hash: str = "c74e97305b95fd6c1a37bbfe65948a8577b0ade967a3fffab5a8d400757c6617"
  status: str = UserPolicyStatus.SUBMITTED
  state_type: str = None

```

In [ ]:
func_plus_one

```python
class SubmitUserCode:
  id: str = None
  code: str = "@sy.syft_function(
    input_policy=ExactMatch(x=x_pointer),
    # input_policy_init_args={'inputs':{'x':4}},
    # output_policy=SingleExecutionExactOutput,
    # output_policy_init_args={},
    output_policy=output_policy.id,
    output_policy_init_args={"n_calls":1, 'downloadable_output_args':['y']},
    outputs=["y"]
)
def func_plus_one(x):
    return {"y": x + 1}
"
  func_name: str = "func_plus_one"
  signature: str = (x)
  input_policy: str = syft.core.node.new.policy.ExactMatch
  input_policy_init_args: str = None
  output_policy: str = 186aff516782421ab982f8198e264ad5
  output_policy_init_args: str = {'n_calls': 1, 'downloadable_output_args': ['y']}
  local_function: str = <function func_plus_one at 0x7f269d9177f0>
  input_kwargs: str = ['x']
  outputs: str = ['y']
  enclave_metadata: str = None

```

In [ ]:
client.api.services.code.func

<function syft.core.node.new.api.generate_remote_function.<locals>.wrapper(*args, **kwargs)>

In [ ]:
client = sy.login(node=worker, email="info@openmined.org", password="changethis", cache=False)

Logged into Eagleman Hutter as <info@openmined.org>


In [ ]:
client.notifications[-1].link.changes[0].link

```python
class UserCode:
  id: str = 5e1e1792fa7944168ca3bf162a4ac58a
  user_verify_key: str = bd2ba16cbdad6886804f76c855f6aca232517b12c6fcb13a51bf782304f1ce19
  raw_code: str = "@sy.syft_function(
    input_policy=ExactMatch(x=x_pointer),
    # input_policy_init_args={'inputs':{'x':4}},
    # output_policy=SingleExecutionExactOutput,
    # output_policy_init_args={},
    output_policy=RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y']),
)
def func(x):
    return {"y": x}
"
  input_policy: str = syft.core.node.new.policy.ExactMatch
  input_policy_state: str = None
  output_policy: str = syft.core.node.new.policy.UserPolicy
  output_policy_state: str = b'\x00\x00\x00\x008\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00%\x00\x00\x00\x16\x00\x00\x00)\x00\x00\x00\x16\x00\x00\x00\x05\x00\x00\x00j\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00syft.core.node.new.policy.RepeatedCallPolicy\x00\x00\x00\x00\r\x00\x00\x00\xca\x00\x00\x00}\x00\x00\x00B\x00\x00\x00\x15\x00\x00\x00\x0e\x00\x00\x00y\x00\x00\x00\x0e\x00\x00\x00downloadable_output_args\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x06\x00\x00\x00\x00\x00\x00\x17\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00r\x00\x00\x00\t\x00\x00\x00\x0e\x00\x00\x00builtins.list\x00\x00\x00\x01\x00\x00\x00\xc2\x03\x00\x00\x00\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x01\x00\x00\x00\x0e\x00\x00\x00\x01\x00\x00\x00\x0e\x00\x00\x00\x01\x00\x00\x00\x82\x02\x00\x00\x00\x00\x00\x00\t\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00j\x00\x00\x00\t\x00\x00\x00\x0e\x00\x00\x00builtins.str\x00\x00\x00\x00\x01\x00\x00\x00\n\x00\x00\x00y\x00\x00\x00\x00\x00\x00\x00n_calls\x00\x01\x00\x00\x00\x82\x02\x00\x00\x00\x00\x00\x00\t\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00j\x00\x00\x00\t\x00\x00\x00\x0e\x00\x00\x00builtins.int\x00\x00\x00\x00\x01\x00\x00\x00\x12\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00'
  parsed_code: str = "

def user_func_func_bd2ba16cbdad6886804f76c855f6aca232517b12c6fcb13a51bf782304f1ce19_49f0ba9524d80f224f3365f64cdc737588aaf4d8429ac80523fe1caa578dc921(x):

    def func(x):
        return {'y': x}
    result = func(x=x)
    return result
"
  service_func_name: str = "func"
  unique_func_name: str = "user_func_func_bd2ba16cbdad6886804f76c855f6aca232517b12c6fcb13a51bf782304f1ce19_49f0ba9524d80f224f3365f64cdc737588aaf4d8429ac80523fe1caa578dc921"
  user_unique_func_name: str = "user_func_func_bd2ba16cbdad6886804f76c855f6aca232517b12c6fcb13a51bf782304f1ce19"
  code_hash: str = "49f0ba9524d80f224f3365f64cdc737588aaf4d8429ac80523fe1caa578dc921"
  signature: str = (x) -> Dict[str, Any]
  status: str = {NodeView(node_name='Eagleman Hutter', verify_key=bd2ba16cbdad6886804f76c855f6aca232517b12c6fcb13a51bf782304f1ce19): <UserCodeStatus.EXECUTE: 'execute'>}
  input_kwargs: str = ['x']
  outputs: str = []
  input_policy_init_args: str = None
  output_policy_init_args: str = {'n_calls': 1, 'downloadable_output_args': ['y']}
  enclave_metadata: str = None

```

In [ ]:
request = client.notifications[-1].link
func = request.changes[0].link
result = func.unsafe_function(x=x_pointer)
# func.output_policy
final_result = request.accept_by_depositing_result(result)
final_result

message='Duplication Key Error: syft.core.node.new.policy.UserPolicy'


<class 'syft.core.node.new.response.SyftSuccess'>: Request 15bf0c7c4d3b488b9ded2293f8171214 changes applied

In [ ]:
client.notifications[-1].link.changes[-1]

```python
class UserCodeStatusChange:
  id: str = 946ca41818034a248db4079b8a65e21c
  linked_obj: str = <<class 'syft.core.node.new.user_code.UserCode'>: 5e1e1792fa7944168ca3bf162a4ac58a@<Node: 4c5dc2d254a1468784643b0ee765cdfd>
  value: str = UserCodeStatus.EXECUTE
  match_type: str = True

```

In [ ]:
client.api.services.code.func_plus_one(x=x_pointer.id)

---------------------------------------------------------------------------
SyftAttributeError
---------------------------------------------------------------------------
Exception: 'APIModule' api.code object has no submodule or method 'func_plus_one', you may not have permission to access the module you are trying to access


In [ ]:
client.api.services.code.get_all()

,type,id,status,service_func_name
0,syft.core.node.new.user_code.UserCode,5e1e1792fa7944168ca3bf162a4ac58a,"{NodeView(node_name='Eagleman Hutter', verify_...",func


In [ ]:
func_plus_one = client.api.services.code.get_all()[-1]

In [ ]:
print(func_plus_one.output_policy.raw_code)

from .user_code import CustomOutputPolicy
@serializable(recursive_serde=True)
class RepeatedCallPolicy(CustomOutputPolicy):
    # from syft.core.node.new.action_object import ActionObject
    __canonical_name__ = "RepeatedCallPolicy"
    from syft.core.node.new.syft_object import SYFT_OBJECT_VERSION_1

    __version__ = SYFT_OBJECT_VERSION_1
    from typing import Dict
    from typing import Any
    n_calls: int
    downloadable_output_args: List[str]
    state: Dict[Any, Any] = {}
    # _init_args: Dict[str, Any]
    # _kwargs: Dict[str, Any]
    
    __attr_allowlist__ = [
        "n_calls",
        "downloadable_output_args",
    ]
    
    # __fields_set__ = [
    #     "n_calls",
    #     "downloadable_output_args",
    # ]
    
    def __init__(self, n_calls=1, downloadable_output_args=[]):
        super().__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_arg

In [ ]:
state = sy.deserialize(func_plus_one.output_policy_state, from_bytes=True, class_type=RepeatedCallPolicy)

In [ ]:
state.public_state()

0